In [1]:
!pip install pandas keras numpy xgboost sklearn nltk textblob

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Get genuine human tweets

In [2]:
import tensorflow as tf
tf.test.gpu_device_name() #We good using GPU

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


''

In [3]:
!pwd

/home/cheriexu/finalcs229


In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, xgboost, numpy as np, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

real_users = pd.read_csv('/home/cheriexu/genuine_accounts/users.csv',low_memory = False)
real_tweets = pd.read_csv('/home/cheriexu/genuine_accounts/tweets.csv',low_memory = False)
real_df = real_tweets.merge(real_users, on='id',how = 'outer', suffixes = ('_tweets','_users')) #Merging the user and tweets
real_df['labels']=pd.Series(np.ones(len(real_df['id']))) #Creating a new column to indicate that this is human data


Using TensorFlow backend.


CANCELED: Get bot data to supplement class imbalance

Import Bot Dataset from this point on 
(https://github.com/fivethirtyeight/russian-troll-tweets/)

In [ ]:
import os
directory = os.fsencode('/home/cheriexu/russian-troll')
troll_data = pd.DataFrame()
for file in os.listdir(directory):
  print(file)
  filename = os.fsdecode(file)
  if filename.endswith(".csv"):
    botdata = pd.read_csv('/home/cheriexu/russian-troll/'+filename,low_memory = False)
    troll_data = troll_data.append(botdata) #append together

In [ ]:
frac = 10
troll_data['labels']=pd.Series(np.zeros(len(botdata)))
#Limiting too only English language tweets
english_bot = troll_data[troll_data['language']  == 'English']
troll_data['language'].unique()
print(len(english_bot),len(troll_data)) #Class imbalance though, we need more spam bot results
train_df = pd.DataFrame() #Create train data frame for splitting later
train_df = train_df.append(real_df[['text','labels']].sample(int(len(real_df)/frac)))
english_bot = english_bot.rename(index=str, columns={"content": "text"})
train_df = train_df.append(english_bot[['text','labels']].sample(int(len(english_bot)/frac)))
train_df['text']= train_df['text'].str.lower()
#Downsample use 10%
len(train_df)

In [ ]:
#troll_data.head()

Split train test dataset:

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], train_df['labels'], test_size = .2)

Create Count Vectors for NB

In [ ]:
# Create CountVectorizer
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
#fill in nan to make sklearn happy
train_df['text'] = train_df['text'].fillna(' ')
train_x = train_x.fillna(' ')
valid_x = valid_x.fillna(' ') 
count_vect.fit(train_df['text'])
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
print(len(train_x),len(valid_x))

In [ ]:
#xtrain_count.shape


Train Model of select type and make predictions. 

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    pred_train = classifier.predict(feature_vector_train)
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.classification_report(label, pred_train), metrics.classification_report(valid_y, predictions)#confusion matrix on validation matrix
  #metrics.confusion_matrix(label, pred_test), metrics.confusion_matrix(valid_y, predictions)

Naive Bayes

In [ ]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
valid_y

In [ ]:
# Naive Bayes on Count Vectors
np.nan_to_num(xtrain_count)
np.nan_to_num(xvalid_count)
conf_test, conf_valid = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print (conf_test[0:2])
print(conf_valid[0:2])

'''

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "NB, N-Gram Vectors: ", accuracy

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
'''

In [ ]:
print (conf_test)
print(conf_valid)

##Naive Bayes on Count Vector Classification Results
###         Train: n = 49596

                  precision   recall  f1-score   support

          0       0.90       0.91      0.90     20069
          1       0.93      0.96      0.95     28448
          
          avg / total       0.90      0.92      0.91     49596
          
###          Valid: n = 12399
  
          
           precision    recall  f1-score   support

          0       0.83      0.85      0.84      5047
          1       0.89      0.91      0.90      7087
    avg / total       0.85      0.87      0.86     12399
          
          
          
          

In [ ]:
#Word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train_df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
np.nan_to_num(xtrain_tfidf)
np.nan_to_num(xvalid_tfidf)
train, test = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)

# Naive Bayes on Tf-iDF
### Train: n = 49596
    precision    recall  f1-score   support

          0       0.85      0.78      0.81     20069
          1       0.85      0.94      0.89     28448
    avg / total       0.83      0.85      0.84     49596
    
###        Valid: n = 12399

    precision    recall  f1-score   support

          0       0.84      0.76      0.80      5047
          1       0.84      0.93      0.88      7087
       avg / total       0.82      0.84      0.83     12399

In [ ]:
print(train)

In [ ]:


# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 